In [ ]:
import os
if os.path.exists("./SPADE"): os.chdir("./SPADE")
    
import pickle
from models.pix2pix_model import Pix2PixModel
from options.test_options import TestOptions
import numpy as np
import IPython.display

import argparse
_ = argparse.ArgumentParser.parse_args
argparse.ArgumentParser.parse_args = lambda self,*args,**kwargs: _(self, args=['--name', 'coco_pretrained', '--dataset_mode', 'coco', '--dataroot', './datasets/coco_stuff'])

opt = TestOptions().parse();
model = Pix2PixModel(opt);
model.eval();

N=182
import base64
from PIL import Image
from io import BytesIO
import torch
import util.util
c = util.util.Colorize(N)
c = c.cmap.numpy()

def generate(src):
    arr = np.array(Image.open(BytesIO(base64.b64decode(src.split(",")[-1]))).resize((256,256)))[:,:,:3]
    globals()["arr"] = 1
    arr2 = np.linalg.norm((c.reshape(N,1,1,3).repeat(256,axis=1).repeat(256,axis=2) - arr),axis=3).argmin(axis=0).reshape(1,1,256,256)
    arr3 = torch.cuda.FloatTensor(1,184,256,256).zero_().scatter_(1,torch.from_numpy(arr2).cuda(),1.0)
    arr4 = util.util.tensor2im(model.netG(arr3)).reshape(256,256,3)
    globals()["arr4"] = arr4
    byte_io = BytesIO()
    Image.fromarray(arr4).save(byte_io, 'PNG')
    dst = b'data:image/png;base64,'+base64.b64encode(byte_io.getvalue())
    return dst

IPython.display.clear_output()
IPython.display.HTML("../main.html")


In [ ]:
# show model parameter
print(model)